In [1]:
%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
from src.blast.complete_blast import blast, DEFAULT_BLAST_ARGS

In /home/NOBACKUP/mzhou3/ming_env/ming_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/NOBACKUP/mzhou3/ming_env/ming_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/NOBACKUP/mzhou3/ming_env/ming_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/NOBACKUP/mzhou3/ming_env/ming_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases

In [2]:
DEFAULT_BLAST_ARGS

{'seed_matching_args': {'seed_matching_algorithm': 'consensus_matching',
  'seed_table_file': '/home/NOBACKUP/mzhou3/comp561project/src/data/hashtable.p',
  'k': 11},
 'ungapped_extension_args': {'nt_score_method': 'sum_proba_score_correct0',
  'delta': 1,
  'mismatch_score': 1,
  'substitution': {}},
 'gapped_extension_args': {'nt_score_method': 'sum_proba_score_correct0',
  'substitution': {},
  'gap_penalty': -1,
  'gap_bias': -1,
  'mismatch_score': 1,
  'ref_max_length_factor': 1}}

# 1. Run blast

`blast(query, reference_matrix_file, seed_matching_args, ungapped_extension_args, gapped_extension_args)`
- query: query string 
- reference_matrix_file: reference matrix file path dict
- seed_matching_args: first stage - seed matching arguments dict
- ungapped_extension_args: second stage - ungapped extension arguments dict
- gapped_extension_args: third stage - gapped extension arguments dict

Default parameters 

In [3]:
DEFAULT_BLAST_ARGS

{'seed_matching_args': {'seed_matching_algorithm': 'consensus_matching',
  'seed_table_file': '/home/NOBACKUP/mzhou3/comp561project/src/data/hashtable.p',
  'k': 11},
 'ungapped_extension_args': {'nt_score_method': 'sum_proba_score_correct0',
  'delta': 1,
  'mismatch_score': 1,
  'substitution': {}},
 'gapped_extension_args': {'nt_score_method': 'sum_proba_score_correct0',
  'substitution': {},
  'gap_penalty': -1,
  'gap_bias': -1,
  'mismatch_score': 1,
  'ref_max_length_factor': 1}}

Run one query using default arguments

In [4]:
query = 'TAAACCCCGGGCCATATTATGCTGGGAGGCTCACCTCTGTAATGTCGGGGTCTTGGAGGCTGAGGAGGTATTGCTTTGGGGAGTTCAAGCCCAGCCTGGG'
reference_matrix_file = 'src/data/reference_matrix.p'
label_pos = 360468

In [5]:
results = blast(query, reference_matrix_file, **DEFAULT_BLAST_ARGS)

In [6]:
for idx, out in enumerate(results):
    print('{0}  ref_start_idx:{1},   ref_end_idx:{2},    score: {3}'.format(idx,
                                                                            out['final_result']['ref_left_idx'],
                                                                            out['final_result']['ref_right_idx'],
                                                                            out['final_result']['score']))


0  ref_start_idx:360468,   ref_end_idx:360567,    score: 45.113333333333344
1  ref_start_idx:360468,   ref_end_idx:360567,    score: 44.37333333333334
2  ref_start_idx:360468,   ref_end_idx:360567,    score: 44.37333333333334
3  ref_start_idx:360468,   ref_end_idx:360567,    score: 43.55333333333334
4  ref_start_idx:360468,   ref_end_idx:360567,    score: 43.27333333333334
5  ref_start_idx:360468,   ref_end_idx:360567,    score: 42.413333333333334
6  ref_start_idx:360468,   ref_end_idx:360567,    score: 42.31333333333334
7  ref_start_idx:264772,   ref_end_idx:264880,    score: -3.8199999999999994
8  ref_start_idx:588359,   ref_end_idx:588459,    score: -7.606666666666663
9  ref_start_idx:588359,   ref_end_idx:588459,    score: -7.606666666666663
10  ref_start_idx:199317,   ref_end_idx:199409,    score: -8.29333333333333
11  ref_start_idx:59115,   ref_end_idx:59208,    score: -8.366666666666662
12  ref_start_idx:566552,   ref_end_idx:566650,    score: -9.046666666666663
13  ref_start_id

# 2. Run individual steps

In [7]:
import pickle
from src.blast.hashtable_generation_inference import consensus_seq_match
from src.blast.ungapped_extension import ungapped_extension
from src.blast.gapped_extension_j_mingmodified import gapped_extension


## 2.1 seed matching

`consensus_seq_match(table_data, query)`
- table_data: seed table data 
- query: the query string 

In [8]:
table_data = pickle.load(open('src/data/hashtable.p','rb'))
outputs_step1 = consensus_seq_match(table_data, query)


## 2.2 ungapped extension

`ungapped_extension(query, matches_dict, reference_matrix_file, k, delta,
                       score_method, mismatch_score=1, substitution=dict())`
- query: query string 
- matches_dict: result from stage 1 
- reference_matrix_file: reference matrix file 
- k: seed size k 
- delta: ungapped extention threshold 
- score_method: nucleotide level scoring function 
- mismatch_score: argument for nucleotide level score 
- substitution: argument for nucleotide level score

In [9]:
outputs_step2 = ungapped_extension(query, outputs_step1, 'src/data/reference_matrix.p', 11, 1, 'sum_proba_score_correct0')
outputs_step2 = sorted(outputs_step2, key=lambda x:x['ungapped_extension_result']['score'] * -1)

for idx, entry in enumerate(outputs_step2):
    
    seed_correct = (entry['seed_matching_result']['query_idx'] + label_pos) == entry['seed_matching_result']['ref_idx']
    score = entry['ungapped_extension_result']['score']
    span = entry['ungapped_extension_result']['query_right_idx']  - entry['ungapped_extension_result']['query_left_idx'] + 1
    print('{3}: seed correct {0},   score {1} ,   span {2}  '.format(seed_correct, float(round(score,2)), span, idx))

0: seed correct True,   score 7.33 ,   span 24  
1: seed correct True,   score 6.42 ,   span 27  
2: seed correct True,   score 5.68 ,   span 27  
3: seed correct True,   score 5.68 ,   span 27  
4: seed correct True,   score 4.86 ,   span 27  
5: seed correct True,   score 4.58 ,   span 27  
6: seed correct True,   score 3.62 ,   span 27  
7: seed correct False,   score 2.92 ,   span 14  
8: seed correct False,   score 2.92 ,   span 14  
9: seed correct False,   score 2.88 ,   span 14  
10: seed correct False,   score 2.88 ,   span 14  
11: seed correct False,   score 2.88 ,   span 14  
12: seed correct False,   score 2.68 ,   span 14  
13: seed correct False,   score 2.64 ,   span 14  
14: seed correct False,   score 2.08 ,   span 14  
15: seed correct False,   score 1.97 ,   span 17  
16: seed correct False,   score 1.59 ,   span 20  
17: seed correct False,   score 0.98 ,   span 12  
18: seed correct False,   score 0.98 ,   span 12  
19: seed correct False,   score 0.98 ,   span 12

## 2.3 gapped extension

`gapped_extension(query, reference, ungapped_dict, score_method,
                     substitution_dict, gap_penalty, gap_bias, mismatch_score, ref_max_length_factor)`
- query: query string
- reference: reference matrix file path 
- ungapped_dict: results from step 2 
- score_method:  nucleotide level scoring method 
- substitution_dict: substitution_dict in NW algorithm 
- gap_penalty:   in NW gap penalty
- gap_bias: in NW open gap penalty
- mismatch_score:   nt level scoring method mis match score 
- ref_max_length_factor:  FACTOR * query length of the reference will be aligned

In [10]:
outputs_step3 = gapped_extension(query, 'src/data/reference_matrix.p', outputs_step2, 'sum_proba_score', {}, -1, -1, 1, 1)


In [11]:
for idx, out in enumerate(outputs_step3):
    print('{0}  ref_start_idx:{1},   ref_end_idx:{2},    score: {3}'.format(idx,
                                                                            out['final_result']['ref_left_idx'],
                                                                            out['final_result']['ref_right_idx'],
                                                                            out['final_result']['score']))


0  ref_start_idx:360468,   ref_end_idx:360567,    score: 42.25333333333334
1  ref_start_idx:360468,   ref_end_idx:360567,    score: 45.013333333333335
2  ref_start_idx:360468,   ref_end_idx:360567,    score: 44.27333333333334
3  ref_start_idx:360468,   ref_end_idx:360567,    score: 44.27333333333334
4  ref_start_idx:360468,   ref_end_idx:360567,    score: 43.45333333333334
5  ref_start_idx:360468,   ref_end_idx:360567,    score: 43.17333333333334
6  ref_start_idx:360468,   ref_end_idx:360567,    score: 42.21333333333334
7  ref_start_idx:500466,   ref_end_idx:500569,    score: -22.753333333333323
8  ref_start_idx:501668,   ref_end_idx:501770,    score: -18.980000000000004
9  ref_start_idx:500466,   ref_end_idx:500569,    score: -22.793333333333326
10  ref_start_idx:501668,   ref_end_idx:501770,    score: -19.020000000000007
11  ref_start_idx:501668,   ref_end_idx:501770,    score: -19.020000000000007
12  ref_start_idx:500466,   ref_end_idx:500569,    score: -22.993333333333325
13  ref_s

# 3. Evaluation full

evaluate single result

In [12]:
from src.blast.eval import compute_performance

`compute_performance(correct_range, outputs)`
- correct_range: correct range of the reference
- outputs: raw outputs from blast algorithm 

In [13]:
correct_range = 360468, 360468+99

In [14]:
compute_performance(correct_range, results)

{'relevant_irelevant_auc': 1.0,
 'top1_iou': 1.0,
 'top5_iou': 1.0,
 'top10_iou': 1.0,
 'seed_match_rate': 0.07777777777777778}